In [1]:
import numpy as np
import pandas as pd 
!pip install psycopg2
import psycopg2 as ps
from io import StringIO


In [2]:
df=pd.read_csv("./Data/customer_support_tickets.csv")

In [3]:
df

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8464,8465,David Todd,adam28@example.net,22,Female,LG OLED,2021-12-08,Product inquiry,Installation support,My {product_purchased} is making strange noise...,Open,NaN,Low,Phone,NaN,NaN,NaN
8465,8466,Lori Davis,russell68@example.com,27,Female,Bose SoundLink Speaker,2020-02-22,Technical issue,Refund request,I'm having an issue with the {product_purchase...,Open,NaN,Critical,Email,NaN,NaN,NaN
8466,8467,Michelle Kelley,ashley83@example.org,57,Female,GoPro Action Camera,2021-08-17,Technical issue,Account access,I'm having an issue with the {product_purchase...,Closed,Eight account century nature kitchen.,High,Social media,2023-06-01 09:44:22,2023-06-01 04:31:22,3.0
8467,8468,Steven Rodriguez,fpowell@example.org,54,Male,PlayStation,2021-10-16,Product inquiry,Payment issue,I'm having an issue with the {product_purchase...,Closed,We seat culture plan.,Medium,Email,2023-06-01 18:28:24,2023-06-01 05:32:24,3.0


In [4]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(
            host=host_name,
            database=dbname,
            user=username,
            password=password,
            port=int(port)  
        )
        print('Connected!')
        return conn
    except Exception as e:
        print(f"Connection error: {e}")
        return None

In [20]:
from dotenv import load_dotenv
import os
import psycopg2 as ps

# Load .env file
load_dotenv()

# Get variables from .env
host_name = os.getenv('host_name')
dbname = os.getenv('dbname')
port = os.getenv('port')
username = os.getenv('user')
password = os.getenv('password')
conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [12]:
from io import StringIO
import pandas as pd
import psycopg2

def push_data_to_db(csv, batch_size=1000):
  
    curr = conn.cursor()
    
   
    df_sample = pd.read_csv(csv, nrows=1)
    columns = []
    for col in df_sample.columns:
        # Make all columns TEXT to avoid numeric conversion issues
        columns.append(f'"{col}" TEXT')
    
    # Create table
    create_table = f"""
    CREATE TABLE IF NOT EXISTS my_table (
        {', '.join(columns)}
    )
    """
    curr.execute(create_table)
    
    # Process data in chunks
    for chunk in pd.read_csv(csv, chunksize=batch_size):
        # Replace NaN with empty string
        chunk = chunk.fillna('')
        
        buffer = StringIO()
        chunk.to_csv(buffer, index=False, header=False, quoting=1)
        buffer.seek(0)
        
        copy_sql = f"""
            COPY my_table ({','.join(f'"{col}"' for col in df_sample.columns)})
            FROM STDIN WITH (FORMAT CSV, QUOTE '"')
        """
        curr.copy_expert(sql=copy_sql, file=buffer)
        conn.commit()
    
    curr.close()
    print("Data uploaded successfully!")


In [13]:

push_data_to_db('./Data/customer_support_tickets.csv', batch_size=1000)

TypeError: push_data_to_db() got an unexpected keyword argument 'batch_size'

In [16]:
cur = conn.cursor()
cur.execute("SELECT * FROM sales_table")

rows = cur.fetchall()
df=pd.DataFrame(rows)
print(df)

        0                    1                           2   3       4   \
0        1        Marisa Obrien  carrollallison@example.com  32   Other   
1        2         Jessica Rios    clarkeashley@example.com  42  Female   
2        3  Christopher Robbins   gonzalestracy@example.com  48   Other   
3        4     Christina Dillon    bradleyolson@example.org  27  Female   
4        5    Alexander Carroll     bradleymark@example.com  67  Female   
...    ...                  ...                         ...  ..     ...   
8464  8465           David Todd          adam28@example.net  22  Female   
8465  8466           Lori Davis       russell68@example.com  27  Female   
8466  8467      Michelle Kelley        ashley83@example.org  57  Female   
8467  8468     Steven Rodriguez         fpowell@example.org  54    Male   
8468  8469      Steven Davis MD          lori20@example.net  53   Other   

                          5           6                7   \
0                 GoPro Hero  2021-03-

In [17]:
cur = conn.cursor()
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'
""")
tables = cur.fetchall()
print("Existing tables:", tables)

Existing tables: [('sales_table',), ('my_table',)]


In [23]:
cur.execute("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = 'my_table'
""")
columns = cur.fetchall()
for col in columns:
    print(f"Column: {col[0]}, Type: {col[1]}")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
